In [2]:
import numpy as np
import pandas as pd

from sklearn import preprocessing

import keras
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from keras import regularizers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score

In [4]:
concrete = pd.read_excel('Concrete_Data.xls')
concrete.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [5]:
concrete.columns

Index(['Cement (component 1)(kg in a m^3 mixture)',
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)',
       'Fly Ash (component 3)(kg in a m^3 mixture)',
       'Water  (component 4)(kg in a m^3 mixture)',
       'Superplasticizer (component 5)(kg in a m^3 mixture)',
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)',
       'Fine Aggregate (component 7)(kg in a m^3 mixture)', 'Age (day)',
       'Concrete compressive strength(MPa, megapascals) '],
      dtype='object')

In [6]:
train_inputs = concrete.drop('Concrete compressive strength(MPa, megapascals) ', axis=1).values
train_target = concrete['Concrete compressive strength(MPa, megapascals) '].values

print(f"{train_inputs.shape}\n{train_target.shape}")

(1030, 8)
(1030,)


In [7]:
train_inputs[:5]

array([[ 540. ,    0. ,    0. ,  162. ,    2.5, 1040. ,  676. ,   28. ],
       [ 540. ,    0. ,    0. ,  162. ,    2.5, 1055. ,  676. ,   28. ],
       [ 332.5,  142.5,    0. ,  228. ,    0. ,  932. ,  594. ,  270. ],
       [ 332.5,  142.5,    0. ,  228. ,    0. ,  932. ,  594. ,  365. ],
       [ 198.6,  132.4,    0. ,  192. ,    0. ,  978.4,  825.5,  360. ]])

In [8]:
train_inputs = preprocessing.scale(train_inputs)
train_inputs[:5]

array([[ 2.47791782, -0.85688631, -0.84713204, -0.91666329, -0.62022529,
         0.86316012, -1.21765845, -0.27973311],
       [ 2.47791782, -0.85688631, -0.84713204, -0.91666329, -0.62022529,
         1.05617041, -1.21765845, -0.27973311],
       [ 0.49144303,  0.79553263, -0.84713204,  2.17536709, -1.03894429,
        -0.52651395, -2.24091255,  3.55306569],
       [ 0.49144303,  0.79553263, -0.84713204,  2.17536709, -1.03894429,
        -0.52651395, -2.24091255,  5.05767679],
       [-0.79043154,  0.67841381, -0.84713204,  0.48880506, -1.03894429,
         0.07053121,  0.64790848,  4.97848673]])

In [9]:
def create_model(learning_rate=0.01, activation='relu'):
    # Use Adam optimizer with the given learning rate
    opt = Adam(learning_rate=learning_rate)
    
    n_cols = train_inputs.shape[1]
    input_shape = (n_cols, )
    
    # create classifier model
    model = Sequential()
    model.add(Dense(units=128, activation=activation, input_shape=input_shape, activity_regularizer=regularizers.l2(1e-5)))
    model.add(Dropout(0.50))
    model.add(Dense(units=128, activation=activation, activity_regularizer=regularizers.l2(1e-5)))
    model.add(Dropout(0.50))
    model.add(Dense(units=1, activation=activation))
    
    # compile the model
    model.compile(optimizer=opt, loss="mae", metrics=['mse', 'mape'])
    return model

In [10]:
# create a KerasRegressor
model = KerasRegressor(build_fn=create_model, verbose=1)

In [11]:
# Define the parameters to tryout
params = {'activation': ['relu', 'tanh'], 'batch_size': [16, 32, 64], 'epochs': [50, 100], 'learning_rate': [0.01, 0.001, 0.0001]}

In [12]:
random_search = RandomizedSearchCV(model, param_distributions=params, cv=KFold(10))

In [ ]:
random_search_results = random_search.fit(train_inputs, train_target)

Epoch 1/100
927/927 [==============================] - 2s 2ms/step - loss: 35.2534 - mse: 1537.8199 - mape: 99.5658
Epoch 2/100
927/927 [==============================] - 0s 417us/step - loss: 35.1393 - mse: 1532.2684 - mape: 99.0039
Epoch 3/100
927/927 [==============================] - 0s 455us/step - loss: 35.0550 - mse: 1526.7340 - mape: 98.5706
Epoch 4/100
927/927 [==============================] - 0s 390us/step - loss: 34.9674 - mse: 1519.9174 - mape: 98.2821
Epoch 5/100
927/927 [==============================] - 0s 423us/step - loss: 34.8744 - mse: 1513.9459 - mape: 97.9132
Epoch 6/100
927/927 [==============================] - 0s 533us/step - loss: 34.7929 - mse: 1507.3374 - mape: 97.5957
Epoch 7/100
927/927 [==============================] - 0s 382us/step - loss: 34.7190 - mse: 1501.3374 - mape: 97.3894
Epoch 8/100
927/927 [==============================] - 0s 437us/step - loss: 34.6485 - mse: 1495.9786 - mape: 97.0996
Epoch 9/100
927/927 [==============================] - 0s 

In [ ]:
print(f"Best Score: {random_search_results.best_score_} and Best Params: {random_search_results.best_params_}")